# Spectral Differentiation of Periodic Functions

Spectral methods are a class of numerical techniques for solving differential equations by expanding the solution in terms of global basis functions, typically trigonometric functions (for periodic problems) or orthogonal polynomials (e.g, Chebyshev polynomials). This approach can achieve extremely high accuracy with relatively function points, particularly when the function is smooth.

In the case of **periodic functions**, spectral differentiation is typically carried out using the **Fourier basis**. Given a periodic function sampled at equispaced points, the derivative can be efficiently computed via the **Fast Fourier Transform (FFT)**. The main idea is to:
1. Transform the function to the frequency domain $u \to \hat{u}$.
2. Differentiate in the frequency domain by multiplying each mode by its wavenumber $\hat{v}_k = \frac{d}{dx} \hat{u} = ik \cdot \hat{u}_k$.
3. Transform back to the physical domain $u' \approx \hat{v} \to v$.

This approach is referred to as **Fourier spectral differentiation**.

#### Benefits

- **High accuracy**: For smooth, periodic functions, spectral methods exhibit **exponential convergence**—the error decreases faster than any power of the grid spacing.
- **Efficiency**: By using the FFT, spectral differentiation is computationally efficient, with $ O(n\log n) $ complexity.
- **Compact representation**: Spectral coefficients often decay rapidly, enabling compact storage and manipulation.

#### Drawbacks

- **Restriction to periodic domains**: Fourier spectral methods assume periodic boundary conditions. Applying them to non-periodic problems often introduces artifacts unless the problem is reformulated (usually what is done to use Chebyshev polynomials).
- **Global nature**: Spectral basis functions are global, meaning that local changes in the solution affect the entire domain, making adaptivity and handling discontinuities difficult.
- **Aliasing and Gibbs phenomena**: Non-smooth functions can suffer from oscillations and loss of accuracy due to high-frequency mode contamination.

### Fourier Interpolant and Connection to FFT/IFFT

Given a $[0, 2\pi]$ periodic function $u(x)$, sampled at $N$ **equispaced points** on the interval $[0, 2\pi)$, we define the grid as:

$$
x_j = \frac{2\pi j}{N}, \quad j = 0, 1, \dots, N-1
$$

These points are **uniformly spaced**, with spacing $h = x_{j+1} - x_j = \frac{2\pi}{N}$.

We define the **Fourier interpolant** $p(x)$ as a trigonometric polynomial that matches the values $ u_j = u(x_j) $ at the grid points. It is given by:

$$
p(x) = \frac{1}{2\pi} \sum_{k = -N/2}^{N/2} \hat{u}_k \, e^{ikx}
$$

where $\hat{u}_k$ are the **Fourier coefficients**, computed as:

$$
\hat{u}_k = \sum_{j=0}^{N-1} u_j \, e^{-ikx_j}, \quad \text{for } k = -N/2, \dots, N/2
$$


#### FFT and IFFT

- **FFT** (Fast Fourier Transform) computes the coefficients $\hat{u}_k$ efficiently from the data $\{u_j\}_{j = 0}^{N-1}$.
- **IFFT** (Inverse FFT) reconstructs the values or interpolant $p(x)$ at the grid points from the Fourier coefficients.

This process allows efficient interpolation and differentiation of periodic functions.




# Examples of Spectral Differentiation of Periodic Functions


In [264]:
using LinearAlgebra
using PlotlyJS
using FFTW
using BenchmarkTools
using Kronecker

1.
$$
u(x) = \exp(\sin(x)),  \quad \text{for } x \in [0, 2\pi)
$$
and we will compare the spectral derivative to its associated first derivative
$$
u'(x) = \cos(x) \exp(\sin(x)) = \cos(x) u(x),  \quad \text{for } x \in [0, 2\pi)
$$

In [265]:
N = 64 # number of points
h = 2 * pi/N # stepsize
x = h * collect(0:N-1) # domain
u = exp.(sin.(x)); # function

In [266]:
# First, we find its fft (Fourier coefficients)
u_hat = fft(u);

In [267]:
# After that, we compute its spectral derivative
v_hat = 1im * (vcat(0:N÷2 - 1, 0, -N÷2 + 1:-1)) .* u_hat; 

In [268]:
# Finally, we compute the ifft to return from the frequency to the physical space
# We will consider only the real part since it f:R->R function
v = real(ifft(v_hat));

In [269]:
# Now, we compare this spectral derivative with the analytical derivative
du_dx = cos.(x) .* u;

In [270]:
# We compute the L_inf error norm between the two derivatives
error = norm(du_dx-v, Inf)/norm(du_dx, Inf)

6.559044768410425e-15

In [271]:
# Also, we will plot the function, its derivative and the residual = du_dx - v
plot([
    scatter(x=x, y=u, mode="markers+lines", name="u"),
    scatter(x=x, y=du_dx, mode="markers+lines", name="du_dx")],
    Layout(xaxis_title="x") #yaxis_title="u(x)",  
)


data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, and xaxis"

In [272]:
plot([
    scatter(x=x, y=du_dx - v, mode="markers+lines")],
    Layout(xaxis_title="x", title="Residual = du_dx - v") #yaxis_title="u(x)",  
)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, and xaxis"

2.
$$
u(x) = \left[\sin^{2}\left(\frac{1}{2}\pi x\right)\right]^{\frac{3}{2}}$$



Unlike our previous function, this function is not $C^{\infty}$. While we can still use spectral differentiation, this reduces the quality of the approximations. Let's start plotting $u(x)$ first.

In [273]:
N = 64 # number of points
u = ((sin.(1/2 * pi * x)).^2).^(3/2); # function

In [274]:
plot([
    scatter(x=x, y=u, mode="markers+lines", name="u")],
    Layout(xaxis_title="x") #yaxis_title="u(x)",  
)

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, and xaxis"

We first notice that the function is periodic in $[0, 2)$. In order to perform spectral derivative, we may consider: 

$$
u(x(z)) = \left[\sin^{2}\left(\frac{1}{2}\pi x(z)\right)\right]^{\frac{3}{2}}$$

in which $x(z) = \frac{2}{2\pi}z.$

Thus, with respect $z$, this function is periodic in $[0, 2\pi)$

In [275]:
N = 64 # number of points
z = h * collect(0:N-1) # domain
x = 1/pi * z
u = ((sin.(1/2 * pi * x)).^2).^(3/2); # function

In [276]:
plot([
    scatter(x=z, y=u, mode="markers+lines", name="u")],
    Layout(xaxis_title="z") #yaxis_title="u(x)",  
)

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, and xaxis"

Now, we may try to compute $\frac{du}{dx}$. We know that 

$$\begin{align*}
\frac{d}{dz} u(x(z)) = \frac{du}{dx} \frac{dx}{dz}
\end{align*}
$$
in which

$$
\frac{du}{dx} = \frac{3}{2}\left[\sin^{2}\left(\frac{1}{2}\pi x(z)\right)\right]^{\frac{1}{2}} 2 \left[\sin\left(\frac{1}{2}\pi x(z)\right)\right]\left[\cos\left(\frac{1}{2}\pi x(z)\right)\right]\left(\frac{1}{2}\pi \right)
$$
$$
\frac{dx}{dz} = \frac{1}{\pi}
$$

If spectral differentation computes $\frac{d}{dz} u(x(z))$, we just need to compute $\frac{du}{dx} = \left( \frac{dx}{dz} \right)^{-1} \frac{d}{dz} u(x(z)).$

In [277]:
# Finding fourier coefficients (or moving to frequency domain)
u_hat = fft(u);

In [278]:
# After that, we compute its spectral derivative
v_hat = 1im * (vcat(0:N÷2 - 1, 0, -N÷2 + 1:-1)) .* u_hat; 

In [279]:
# Finally, we compute the ifft to return from the frequency to the physical space
# We will consider only the real part since it f:R->R function
v = real(ifft(v_hat));

In [280]:
# Finding du/dx = (dx/dz)^-1 * d/dz u(x(z)) 
dx_dz = 1/pi
v = v * (dx_dz)^(-1);

In [281]:
# Finding analytical derivative on domain
du_dx = 3 .* abs.(sin.(0.5π .* x)) .* sin.(0.5π .* x) .* cos.(0.5π .* x) .* (0.5π);

In [282]:
# We compute the L_inf error norm between the two derivatives
error = norm(du_dx-v, Inf)/norm(du_dx, Inf)

0.00023756243335325665

We notice that the error is much larger this time, since $u(x)$ is not as smooth as the previous example. 

# Solution of differential equations

TBD